In [0]:
import pandas as pd
import tensorflow as tf
import os
import re
import numpy as np
from string import punctuation
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
PATH='/content/gdrive/My Drive/ML Project/train.csv'

In [4]:
data = pd.read_csv(PATH)
print('총 샘플의 수 :',len(data))
train_data=data.copy()
train_data['title'].nunique(), data['text'].nunique() #기사제목, 기사내용 중복 내용 확인
# 윈도우 바탕화면에서 작업한 저자의 경우에는
# data = pd.read_csv(r'C:\Users\USER\Desktop\spam.csv',encoding='latin1')
train_data.drop_duplicates(subset=['text'], inplace=True) # v2 열에서 중복인 내용이 있다면 중복 제거
print('중복 제거 후 샘플의 수 :',len(train_data))

총 샘플의 수 : 20800
중복 제거 후 샘플의 수 : 20387


In [0]:
# importing neural network libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, LSTM, RNN, SpatialDropout1D
from tensorflow.keras.models import load_model

In [0]:
train_data = train_data.set_index('id', drop = True)

In [7]:
print(train_data.shape)
train_data.head()

(20387, 4)


,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
train_data.isnull().sum()

title      518
author    1854
text         1
label        0
dtype: int64

In [9]:
train_data[['title', 'author']] = train_data[['title', 'author']].fillna(value = 'Missing')
train_data = train_data.dropna()
train_data.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [10]:
length = []
[length.append(len(str(text))) for text in train_data['text']]
train_data['length'] = length
train_data.head()

,title,author,text,label,length
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,4930
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,4160
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,7692
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,3237
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,938


In [11]:
train_data = train_data.drop(train_data['text'][train_data['length'] < 50].index, axis = 0)
print('기사의 최대길이 : %d' % max(j for j in train_data['length']))

기사의 최대길이 : 142961


In [0]:
max_features = 4500

In [0]:
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = train_data['text'])
X = tokenizer.texts_to_sequences(texts = train_data['text'])

In [14]:
print(X[20000])

[210, 145, 22, 438, 2, 38, 3377, 676, 86, 31, 19, 1, 1170, 5, 896, 20, 3126, 598, 28, 17, 817, 5, 630, 3, 1, 2749, 491, 3, 1, 732, 715, 1491, 17, 210, 145, 22, 438, 2, 21, 29, 1910, 20, 103, 1242, 3, 9, 1275, 580, 85, 55, 1, 823, 44, 22, 438, 2, 2712, 525, 3, 1, 48, 65, 17, 7, 1910, 145, 22, 966, 778, 236, 1, 205, 2, 147, 5, 2354, 37, 60, 28, 47, 967, 1401, 6, 1040, 7, 29, 2032, 311, 3, 210, 19, 1376, 676, 86, 31, 77, 24, 163, 1693, 896, 2, 70, 2965, 3844, 2, 567, 907, 1, 1170, 3, 38, 3377, 303, 817, 154, 1, 2023, 10, 586, 3595, 3, 27, 1230, 3, 3581, 70, 1231, 1046, 567, 1, 490, 1, 1170, 3, 1, 51, 8, 529, 49, 658, 4, 49, 58, 48, 145, 21, 7, 266, 485, 54, 5, 490, 221, 24, 163, 2, 21, 5, 33, 8, 43, 1170, 37, 1, 69, 3417, 49, 2, 179, 143, 12, 1, 8, 24, 122, 2, 22, 189, 9, 1, 823, 4, 99, 24, 122, 2, 22, 189, 9, 1, 2112, 224, 11, 48, 1329, 1314, 3403, 64, 5, 787, 1997, 1910, 8, 24, 86, 7, 91, 2, 48, 112, 151, 10, 97, 1095, 1997, 1, 823, 8, 24, 231, 5, 425, 119, 8, 5, 768, 337, 193, 20, 7, 1

In [15]:
X = pad_sequences(sequences = X, maxlen = max_features, padding = 'pre')
print(X.shape)

(20263, 4500)


In [16]:
print(X.shape)
y = train_data['label'].values
print(y.shape)
print(X)
m=len(X)

(20263, 4500)
(20263,)
[[   0    0    0 ... 1096    5 4201]
 [   0    0    0 ... 3733  139 1166]
 [   0    0    0 ...    3  207  671]
 ...
 [   0    0    0 ... 3741  778   11]
 [   0    0    0 ...   10   59 3805]
 [   0    0    0 ... 1565 2266 3643]]


In [0]:
n_of_train = int(len(X) * 0.8)
n_of_test = int(len(X) - n_of_train)

In [0]:
model = Sequential()
model.add(Embedding(m, 32))
model.add(GRU(32))
model.add(Dense(1, activation='sigmoid'))

In [0]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('/content/gdrive/My Drive/ML Project/GRUbest_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [20]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X, y, epochs=30, batch_size=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/30
1621/1621 [==============================] - ETA: 0s - loss: 0.2732 - acc: 0.8901
Epoch 00001: val_acc improved from -inf to 0.93511, saving model to /content/gdrive/My Drive/ML Project/GRUbest_model.h5
1621/1621 [==============================] - 208s 128ms/step - loss: 0.2732 - acc: 0.8901 - val_loss: 0.1779 - val_acc: 0.9351
Epoch 2/30
1621/1621 [==============================] - ETA: 0s - loss: 0.1592 - acc: 0.9482
Epoch 00002: val_acc improved from 0.93511 to 0.94498, saving model to /content/gdrive/My Drive/ML Project/GRUbest_model.h5
1621/1621 [==============================] - 206s 127ms/step - loss: 0.1592 - acc: 0.9482 - val_loss: 0.1715 - val_acc: 0.9450
Epoch 3/30
1621/1621 [==============================] - ETA: 0s - loss: 0.1182 - acc: 0.9613
Epoch 00003: val_acc improved from 0.94498 to 0.96447, saving model to /content/gdrive/My Drive/ML Project/GRUbest_model.h5
1621/1621 [==============================] - 206s 127ms/step - loss: 0.1182 - acc: 0.9613 - val_lo

In [21]:
loaded_model = load_model('/content/gdrive/My Drive/ML Project/GRUbest_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X, y)[1]))

634/634 [==============================] - 37s 59ms/step - loss: 0.0226 - acc: 0.9934

 테스트 정확도: 0.9934
